In [ ]:
import os
import random
from pprint import pprint

import dotenv
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.example_selectors.base import BaseExampleSelector
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    PromptTemplate,
)
from pydantic import BaseModel, Field, ValidationError


from tools.data_manager import CupaDatamanager
from tools.constants import WRITE_DIR, DICT_CEFR_DESCRIPTIONS

# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../.env", override=True)

In [ ]:
cupa_data_dir = "../data/raw/CUPA"
cupa_dm = CupaDatamanager()
dataset = cupa_dm.get_cupa_dataset(cupa_data_dir, WRITE_DIR, save_dataset=False)

In [ ]:
# load data
dataset["train"]

In [ ]:
def human_format_input(row) -> str:
    return f"Context:\n{row.context}\nQuestion: {row.question}\nOptions:\n1. {row.option_0}\n2. {row.option_1}\n3. {row.option_2}\n4. {row.option_3}"

def human_format_output(row) -> str:
    return f"Question difficulty: {row.difficulty}"

In [ ]:
examples_df = pd.DataFrame()
examples_df["input"] = dataset["train"].apply(human_format_input, axis=1)
examples_df["output"] = dataset["train"].apply(human_format_output, axis=1)
examples_df.head()

In [ ]:
# try with first 20 examples
examples_df = examples_df.head(20)
few_shot_list = [{"input": row["input"], "output": row["output"]} for _, row in examples_df.iterrows()]

# Dynamic few-shot prompting

## Create example selector

NOTE: I need OpenAI credits to use the OpenAI embeddings.

In [ ]:
# examples = few_shot_list
# to_vectorize = [" ".join(example.values()) for example in examples]
# embeddings = OpenAIEmbeddings()
# vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [ ]:
# example_selector = SemanticSimilarityExampleSelector(
#     vectorstore=vectorstore,
#     k=2,
# )

# # The prompt template will load examples by passing the input do the `select_examples` method
# example_selector.select_examples({"input": "horse"})

In [ ]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        random_match = random.choice(self.examples)
        return [random_match]

example_selector = RandomExampleSelector(examples=few_shot_list)
example_selector.select_examples({})

## Create prompt template

In [ ]:
system_prompt_template = PromptTemplate.from_template(
    "You are a student working on {exam_type}, containing multiple choice questions. "
    "You will be asked to provide the difficulty level of the question."
)

system_prompt_input = system_prompt_template.format(exam_type="an English reading comprehension exam")
system_prompt_input

In [ ]:
# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?").to_messages())

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_input),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.invoke(input="What's 3 🦜 3?"))

## Use with a chat model

In [ ]:
test_example = {"input": examples_df.loc[21, "input"]}
test_example

In [ ]:
chain = final_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain.invoke(test_example)

___
Ask to find zero-shot misconceptions

In [ ]:
system_prompt_template = PromptTemplate.from_template(
    "You are a student working on {exam_type}, containing multiple choice questions. "
    "You are currently at CEFR level {level}, which means that you {level_description} "
    "What misconceptions could you have about the question that could lead to answering it incorrectly? "
    "For each answer option, explain the possible misconception that could lead to selecting that option. "
    "Furthermore, provide the answer that you think is correct (as an integer in the range 1-4)."
)

system_prompt_input = system_prompt_template.format(
    exam_type="an English reading comprehension exam",
    level="B2",
    level_description=DICT_CEFR_DESCRIPTIONS["B2"],
)
system_prompt_input

In [ ]:
test_example = {"input": examples_df.loc[21, "input"]}
test_example

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_input),
        ("human", "{input}"),
    ]
)

print(final_prompt.invoke(input="What's 3 🦜 3?"))

In [ ]:
# Pydantic
class MCQAnalysis(BaseModel):
    """Analysis of an multiple-choice question."""

    option_1: str = Field(description="Misconception in option 1")
    option_2: str = Field(description="Misconception in option 2")
    option_3: str = Field(description="Misconception in option 3")
    option_4: str = Field(description="Misconception in option 4")
    student_answer: int = Field(
        description="The student's answer to the question, as an integer (1-4)"
    )
    # difficulty: str = Field(description="The difficulty level of the question")

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0).with_structured_output(
    MCQAnalysis
)
chain = final_prompt | model

test_output = chain.invoke(test_example)
test_output

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
    temperature=0.1,
).with_structured_output(MCQAnalysis)
chain = final_prompt | model

test_output = chain.invoke(test_example)
test_output